In [ ]:
# !pip install deepface opencv-python-headless

from deepface import DeepFace
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
from google.colab import files

# ============================
# 1️⃣ Upload photo
# ============================
uploaded = files.upload()
image_path = list(uploaded.keys())[0]

# ============================
# 2️⃣ Load and Preprocess Image
# ============================
img = cv2.imread(image_path)


#pratik--> my added code for color conversion
img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# def preprocess_image(frame):
'''    # --- Convert to RGB ---
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # --- Resize if too large (helps detection) ---
    h, w = frame.shape[:2]
    if w > 1280:
        frame = cv2.resize(frame, (1280, int(h * (1280 / w))))

    # --- Brightness & contrast adjustment ---
    # frame = cv2.convertScaleAbs(frame, alpha=0.8, beta=25)

    # --- Denoise slightly (removes CCTV compression noise) ---
    frame = cv2.fastNlMeansDenoisingColored(frame, None, 10, 10, 7, 21)

    # --- Sharpening filter (helps edge clarity for faces) ---
    kernel = np.array([[0, -1, 0],
                       [-1, 5, -1],
                       [0, -1, 0]])
    frame = cv2.filter2D(frame, -1, kernel)

    # --- Optional: Histogram equalization (brightness normalization) ---
    lab = cv2.cvtColor(frame, cv2.COLOR_RGB2LAB)
    l, a, b = cv2.split(lab)
    l = cv2.equalizeHist(l)
    lab = cv2.merge((l, a, b))
    frame = cv2.cvtColor(lab, cv2.COLOR_LAB2RGB)
'''
  # return frame

# Preprocess the uploaded image
processed_img = img

# ============================
# 3️⃣ Detect faces using multiple detectors
# ============================
detectors = ['retinaface', 'mtcnn', 'mediapipe', 'ssd', 'opencv']
all_faces = []

for backend in detectors:
    try:
        print(f"Trying detector: {backend}")
        detections = DeepFace.extract_faces(
            img_path=processed_img,
            detector_backend=backend,
            enforce_detection=False
        )
        for face in detections:
            region = face['facial_area']
            all_faces.append(region)
    except Exception as e:
        print(f"Error with {backend}: {e}")

# ============================
# 4️⃣ Remove near-duplicate faces
# ============================
unique_faces = []
for region in all_faces:
    if not any(abs(region['x'] - uf['x']) < 15 and abs(region['y'] - uf['y']) < 15 for uf in unique_faces):
        unique_faces.append(region)

# ============================
# 5️⃣ Draw bounding boxes
# ============================
output = processed_img.copy()
for region in unique_faces:
    x, y, w, h = region['x'], region['y'], region['w'], region['h']
    cv2.rectangle(output, (x, y), (x+w, y+h), (0, 255, 0), 2)

# ============================
# 6️⃣ Display result
# ============================
cv2_imshow(cv2.cvtColor(output, cv2.COLOR_RGB2BGR))
print(f"✅ Total unique faces detected: {len(unique_faces)}")
